# Exploring New (To Me) Mediapipe Workflow

Instead of prior workflow of doing pose estimation, calculations, and display all in one loop, trying a new workflow of doing pose estimations in one loop, then doing calculations, then doing a final loop for displaying and saving the edited video.

### Imports

In [1]:
# imports
import cv2
import mediapipe as mp
import numpy as np
import math

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


### Define Functions 

In [2]:
### center of mass functions - eventually we would make these a class
def calculateCOM(proximal, distal, com_proximal_multiplier):
    segment_length = distal-proximal
    segment_COM = proximal + (com_proximal_multiplier*segment_length)
    return segment_COM

def calculate_foot_COM(heel, ankle, foot_index):
    COM_foot = (heel + ankle + foot_index)/3
    return COM_foot

def calculate_hand_COM(wrist, index, pinky, com_proximal_multiplier):
    knuckle_width = pinky - index
    Third_metacarple = index + (knuckle_width/3)
    Palm_segment = Third_metacarple - wrist
    COM_hand = wrist + (com_proximal_multiplier*Palm_segment)
    return COM_hand

def calculate_total_COM(COM_Segments, dimension):
    '''Calculate total body COM based on COM of all segments, separately for x and y. 
    Dimension should be 0 for x and 1 for y.'''

    return sum([COM_Segments[segment][2]*COM_Segments[segment][dimension] for segment in COM_Segments])

def COM_finite_difference(current, previous, fps):
    return (current - previous) * fps #we multiply change by fps to find change over time because fps is inverse of change in time

# Do Pose Estimation and Save Results

### Setup Capture

In [3]:
#set file path to the video location
path = "/Users/Philip/Documents/GitHub/Learning/Videos/delaney_almighty.mp4"

#capture the video using opencv
cap = cv2.VideoCapture(path)

### Get capture properties

We'll use these later when it comes time to display the COM and also to write the video

In [4]:
#setup properties for video writer
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

Create array to store pose estimation results, and run the pose estimation loop

In [5]:
results_array = []

In [6]:
with mp_pose.Pose(model_complexity = 2, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))): #loop through feed
        ret, frame = cap.read() #getting an image from feed, 'frame' is our video feed variable
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) #recolor image into the RGB format (for mediapipe)
        image.flags.writeable = False
        
        #make pose detection in MediaPipe
        results = pose.process(image)
       
        results_array.append(results)
        
        image.flags.writeable = True #setting this to true allows the drawing of the landmarks onto the image
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #recolor image back to BGR (for opencv)
        
        cv2.imshow('COM Display (q to quit)', image)

        if cv2.waitKey(10) & 0xFF == ord('q'): #break out of feed by typing 'q' key
            break

        
cv2.destroyAllWindows()
cv2.waitKey(1) #helps close windows for certain mac users
#release camera and close all windows
cap.release()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Now that our loop has run, we can investigate what we get out. It looks like we have an array of 403 objects from the mediapipe SolutionOutputs class.

In [14]:
print(len(results_array))
print(results_array[0])

403
<class 'mediapipe.python.solution_base.SolutionOutputs'>


From these solutionsOutput objects, we can extract landmarks just like we did before in the for loop

In [9]:
landmarks = results_array[0].pose_landmarks.landmark
print(landmarks)

[x: 0.624949038028717
y: 0.2391638308763504
z: 0.016450844705104828
visibility: 0.999797523021698
, x: 0.6227338314056396
y: 0.22962228953838348
z: 0.006337265949696302
visibility: 0.9997367262840271
, x: 0.6198492646217346
y: 0.2286323755979538
z: 0.005942911375313997
visibility: 0.9997760653495789
, x: 0.6168407797813416
y: 0.22782078385353088
z: 0.006051038857549429
visibility: 0.9997811913490295
, x: 0.6285052299499512
y: 0.23213335871696472
z: -0.03414587676525116
visibility: 0.9998446702957153
, x: 0.6297038197517395
y: 0.23283985257148743
z: -0.03464222699403763
visibility: 0.9999086856842041
, x: 0.6309459805488586
y: 0.23350703716278076
z: -0.03486674278974533
visibility: 0.9999065399169922
, x: 0.5995835661888123
y: 0.2294577658176422
z: -0.0023981642443686724
visibility: 0.9998695850372314
, x: 0.6239383220672607
y: 0.23662881553173065
z: -0.1874236911535263
visibility: 0.9998613595962524
, x: 0.6136713027954102
y: 0.24545760452747345
z: 0.02821359969675541
visibility: 0.999

Now let's try getting hand COM out, which should look like all of our prior code from the COM script, just without the display step

# Get COM Data

In [17]:
# COM of Hands, creating third metacarple point by taking 1/3 of the distance of the segment between index and pinky landmarks
# then adding that distance to the index. Palm segment will be from wrist joint to 3rd metacarple joint.
# dictionary values arranged in [wrist, index, pinky, com_proximal_multiplier, com_x, com_y] in that order. 
Hands = {
    'L_hand' : [landmarks[15], landmarks[19], landmarks[17], 0.7474, 0, 0],
    'R_hand' : [landmarks[16],landmarks[20], landmarks[18], 0.7474, 0, 0],
}
          
for key in Hands:
    #Knuckle_width is defined as the segment from the distal ends of the second to fifth metacarples (index to pinky landmarks)
    wrist_x = Hands[key][0].x
    wrist_y = Hands[key][0].y
    index_x = Hands[key][1].x
    index_y = Hands[key][1].y
    pinky_x = Hands[key][2].x
    pinky_y = Hands[key][2].y
    com_proximal_multiplier = Hands[key][3] 

    COM_hand_x = calculate_hand_COM(wrist_x, index_x, pinky_x, com_proximal_multiplier)
    COM_hand_y = calculate_hand_COM(wrist_y, index_y, pinky_y, com_proximal_multiplier)

    #update Hands dictionary with COM positions
    Hands[key][4] = COM_hand_x
    Hands[key][5] = COM_hand_y
    
print(Hands)

{'L_hand': [x: 0.637825608253479
y: 0.14012941718101501
z: 0.0856127142906189
visibility: 0.9780105948448181
, x: 0.6479474306106567
y: 0.12289657443761826
z: 0.03733926638960838
visibility: 0.9751825332641602
, x: 0.6365339756011963
y: 0.1238325834274292
z: 0.07487407326698303
visibility: 0.9759303331375122
, 0.7474, 0.6425471861918768, 0.12748278155426185], 'R_hand': [x: 0.8419061303138733
y: 0.28335079550743103
z: -0.5047881603240967
visibility: 0.9926447868347168
, x: 0.870241105556488
y: 0.27091628313064575
z: -0.5525856018066406
visibility: 0.9900314807891846
, x: 0.8714683651924133
y: 0.27848756313323975
z: -0.5870994329452515
visibility: 0.9912751913070679
, 0.7474, 0.8633894420941671, 0.275943499181668]}


There we go! We have our left and right hand COM stored nicely in a dictionary, just like before. We should probably move this, and the other COM calculations, into functions, since we're going to be calling them 400+ times for our short video.

In [18]:
def handCOM(landmarks):
    # COM of Hands, creating third metacarple point by taking 1/3 of the distance of the segment between index and pinky landmarks
    # then adding that distance to the index. Palm segment will be from wrist joint to 3rd metacarple joint.
    # dictionary values arranged in [wrist, index, pinky, com_proximal_multiplier, com_x, com_y] in that order. 
    Hands = {
        'L_hand' : [landmarks[15], landmarks[19], landmarks[17], 0.7474, 0, 0],
        'R_hand' : [landmarks[16],landmarks[20], landmarks[18], 0.7474, 0, 0],
    }

    for key in Hands:
        #Knuckle_width is defined as the segment from the distal ends of the second to fifth metacarples (index to pinky landmarks)
        wrist_x = Hands[key][0].x
        wrist_y = Hands[key][0].y
        index_x = Hands[key][1].x
        index_y = Hands[key][1].y
        pinky_x = Hands[key][2].x
        pinky_y = Hands[key][2].y
        com_proximal_multiplier = Hands[key][3] 

        COM_hand_x = calculate_hand_COM(wrist_x, index_x, pinky_x, com_proximal_multiplier)
        COM_hand_y = calculate_hand_COM(wrist_y, index_y, pinky_y, com_proximal_multiplier)

        #update Hands dictionary with COM positions
        Hands[key][4] = COM_hand_x
        Hands[key][5] = COM_hand_y
    return Hands
    

In [26]:
Hands = handCOM(landmarks)

Great, our hand function worked smoothly, let's add the others in too

In [20]:
def feetCOM(landmarks):
    #COM of FEET, using the centroid of the triangle formed from heel-ankle-"foot-index" landmarks, arranged in that order in the dictionary values.
    Feet = {
        'L_foot' : [landmarks[29], landmarks[27], landmarks[31], 0, 0],
        'R_foot' : [landmarks[30], landmarks[28], landmarks[32], 0, 0]
    }

    for key in Feet:
        #arguments are ordered heel, ankle, foot index
        COM_foot_x = calculate_foot_COM(Feet[key][0].x, Feet[key][1].x, Feet[key][2].x)
        COM_foot_y = calculate_foot_COM(Feet[key][0].y, Feet[key][1].y, Feet[key][2].y)

        #plot feet COM
        cv2.circle(image, center=tuple(np.multiply((COM_foot_x, COM_foot_y), [width, height]).astype(int)), radius=1, color=(255,0,0), thickness=3)

        #update Feet dictionary with COM positions
        Feet[key][3] = COM_foot_x
        Feet[key][4] = COM_foot_y
        
    return Feet

In [27]:
Feet = feetCOM(landmarks)

In [22]:
def trunkCOM(landmarks):
    Trunk = []
    
    #Trunk segment calculations from MidShoulder to Mid Hip. 
    MidShoulder_x = (landmarks[12].x + landmarks[11].x)/2
    MidShoulder_y = (landmarks[12].y + landmarks[11].y)/2
    MidHip_x = (landmarks[24].x + landmarks[23].x)/2
    MidHip_y = (landmarks[24].y + landmarks[23].y)/2
    TrunkCOM_x = calculateCOM(MidShoulder_x, MidHip_x, 0.3782)
    TrunkCOM_y = calculateCOM(MidShoulder_y, MidHip_y, 0.3782)
    
    Trunk.append(TrunkCOM_x)
    Trunk.append(TrunkCOM_y)
    
    return Trunk

In [28]:
Trunk = trunkCOM(landmarks)

In [24]:
 def body_segmentsCOM(landmarks):
    #Body Segment Dictionary format: key = body segment, % value = [proximal joint landmark values, distal joint landmark values, COM as a % of segment length]
    Body_Segments = {
        'L_UpperArm' : [landmarks[11] , landmarks[13] , 0.5754, 0, 0],
        'R_UpperArm' : [landmarks[12] , landmarks[14] , 0.5754, 0, 0], 
        'L_Forearm' : [landmarks[13] , landmarks[15] , 0.4559, 0, 0],
        'R_Forearm' : [landmarks[14] , landmarks[16] , 0.4559, 0, 0], 
        'L_Thigh' : [landmarks[23], landmarks[25], 0.3612, 0, 0], 
        'R_Thigh' : [landmarks[24], landmarks[26], 0.3612, 0, 0],
        'L_Shin' : [landmarks[25], landmarks[27], 0.4416, 0, 0],
        'R_Shin' : [landmarks[26], landmarks[28], 0.4416, 0, 0],
        'Head' : [landmarks[7], landmarks[8], 0.5, 0, 0]
    }


    for key in Body_Segments:
        #print(key, 'proximal x ->', Body_Segments[key][0].x) 
        x1 = Body_Segments[key][0].x #proximal joint x value
        y1 = Body_Segments[key][0].y
        x2 = Body_Segments[key][1].x
        y2 = Body_Segments[key][1].y
        com_proximal_multiplier = Body_Segments[key][2]

        COM_x = calculateCOM(x1, x2, com_proximal_multiplier)
        COM_y = calculateCOM(y1, y2, com_proximal_multiplier)
        
        Body_Segments[key][3] = COM_x
        Body_Segments[key][4] = COM_y
        
    return Body_Segments


In [35]:
Body_Segments = body_segmentsCOM(landmarks)

In [30]:
def define_COM_segments(Body_Segments, Trunk, Feet, Hands):
    COM_Segments = {
                'Head' : [ Body_Segments['Head'][3], Body_Segments['Head'][4], 0.0668],
                'Trunk' : [ Trunk[0], Trunk[1], 0.4257 ],
                'Left_Upper_Arm' : [ Body_Segments['L_UpperArm'][3], Body_Segments['L_UpperArm'][4], 0.0255],
                'Right_Upper_Arm' : [ Body_Segments['R_UpperArm'][3], Body_Segments['R_UpperArm'][4], 0.0255],
                'Left_Forearm' : [ Body_Segments['L_Forearm'][3], Body_Segments['L_Forearm'][4], 0.0138],
                'Right_Forearm' : [ Body_Segments['R_Forearm'][3], Body_Segments['R_Forearm'][4], 0.0138],
                'Left_Hand' : [Hands['L_hand'][4], Hands['L_hand'][5], 0.0056],
                'Right_Hand' : [Hands['R_hand'][4], Hands['R_hand'][5], 0.0056],
                'Left_Thigh' : [Body_Segments['L_Thigh'][3], Body_Segments['L_Thigh'][4], 0.1478],
                'Right_Thigh' : [Body_Segments['R_Thigh'][3], Body_Segments['R_Thigh'][4], 0.1478],
                'Left_Shin' : [Body_Segments['L_Shin'][3], Body_Segments['L_Shin'][4], 0.0481],
                'Right_Shin' : [Body_Segments['R_Shin'][3], Body_Segments['R_Shin'][4], 0.0481],
                'Left_Foot' : [Feet['L_foot'][3], Feet['L_foot'][4], 0.0129], 
                'Right_Foot' : [Feet['R_foot'][3], Feet['R_foot'][4], 0.0129], 
            }
    return COM_Segments

In [37]:
COM_Segments = define_COM_segments(Body_Segments, Trunk, Feet, Hands)
print(COM_Segments)

{'Head': [0.6117609441280365, 0.23304329067468643, 0.0668], 'Trunk': [0.5411405823767186, 0.3120762170881033, 0.4257], 'Left_Upper_Arm': [0.5722999109745026, 0.21366205137968064, 0.0255], 'Right_Upper_Arm': [0.6790794296979904, 0.28986579669713974, 0.0255], 'Left_Forearm': [0.6134688444375992, 0.16720359746813773, 0.0138], 'Right_Forearm': [0.7798686645448207, 0.2904709319353104, 0.0138], 'Left_Hand': [0.6425471861918768, 0.12748278155426185, 0.0056], 'Right_Hand': [0.8633894420941671, 0.275943499181668, 0.0056], 'Left_Thigh': [0.4734547832608223, 0.4221145317435265, 0.1478], 'Right_Thigh': [0.5042994261980057, 0.43415021011829374, 0.1478], 'Left_Shin': [0.5006555877208709, 0.5247271077156067, 0.0481], 'Right_Shin': [0.4915781452178955, 0.5401873030662536, 0.0481], 'Left_Foot': [0.5112598737080892, 0.5903655290603638, 0.0129], 'Right_Foot': [0.4746156136194865, 0.6074244578679403, 0.0129]}


In [38]:
def get_total_COM(COM_Segments):
    COM_total_x = calculate_total_COM(COM_Segments, 0) #0 points to x dimension
    COM_total_y = calculate_total_COM(COM_Segments, 1) #1 points to y dimension
    
    return [COM_total_x, COM_total_y]

In [39]:
total_COM = get_total_COM(COM_Segments)
total_COM

[0.5357569614139249, 0.363062996072482]

In [43]:
def update_COM_Segments(COM_Segments, total_COM):
    COM_Segments['Total'] = total_COM

In [45]:
update_COM_Segments(COM_Segments, total_COM)
print(COM_Segments)

{'Head': [0.6117609441280365, 0.23304329067468643, 0.0668], 'Trunk': [0.5411405823767186, 0.3120762170881033, 0.4257], 'Left_Upper_Arm': [0.5722999109745026, 0.21366205137968064, 0.0255], 'Right_Upper_Arm': [0.6790794296979904, 0.28986579669713974, 0.0255], 'Left_Forearm': [0.6134688444375992, 0.16720359746813773, 0.0138], 'Right_Forearm': [0.7798686645448207, 0.2904709319353104, 0.0138], 'Left_Hand': [0.6425471861918768, 0.12748278155426185, 0.0056], 'Right_Hand': [0.8633894420941671, 0.275943499181668, 0.0056], 'Left_Thigh': [0.4734547832608223, 0.4221145317435265, 0.1478], 'Right_Thigh': [0.5042994261980057, 0.43415021011829374, 0.1478], 'Left_Shin': [0.5006555877208709, 0.5247271077156067, 0.0481], 'Right_Shin': [0.4915781452178955, 0.5401873030662536, 0.0481], 'Left_Foot': [0.5112598737080892, 0.5903655290603638, 0.0129], 'Right_Foot': [0.4746156136194865, 0.6074244578679403, 0.0129], 'Total': [0.5357569614139249, 0.363062996072482]}


Great! We're getting our total COM out just by running a bunch of functions, and we've added it to our dictionary so all of our COM data is in one place. Now let's make a function that does all of the above, so we can iterate through our video frames and get a list of completed COM dictionaries out. 

In [46]:
def get_COM_dict(landmarks):
    Hands = handCOM(landmarks)
    Feet = feetCOM(landmarks)
    Trunk = trunkCOM(landmarks)
    Body_Segments = body_segmentsCOM(landmarks)
    COM_Segments = define_COM_segments(Body_Segments, Trunk, Feet, Hands)
    total_COM = get_total_COM(COM_Segments)
    update_COM_Segments(COM_Segments, total_COM)
    return COM_Segments

And now that we have a function, we can iterate through all of our results to get the COM data for the whole video

In [103]:
whole_video_COM = []
for result in results_array:
    try:
        COM_dict = get_COM_dict(result.pose_landmarks.landmark)
        whole_video_COM.append(COM_dict)
    except:
        #if pose isn't detected, we fill out dictionary with NotANumber values
        nan_dict = {'Head': [math.nan, math.nan, math.nan],
         'Trunk': [math.nan, math.nan, math.nan],
         'Left_Upper_Arm': [math.nan, math.nan, math.nan],
         'Right_Upper_Arm': [math.nan, math.nan, math.nan],
         'Left_Forearm': [math.nan, math.nan, math.nan],
         'Right_Forearm': [math.nan, math.nan, math.nan],
         'Left_Hand': [math.nan, math.nan, math.nan],
         'Right_Hand': [math.nan, math.nan, math.nan],
         'Left_Thigh': [math.nan, math.nan, math.nan],
         'Right_Thigh': [math.nan, math.nan, math.nan],
         'Left_Shin': [math.nan, math.nan, math.nan],
         'Right_Shin': [math.nan, math.nan, math.nan],
         'Left_Foot': [math.nan, math.nan, math.nan],
         'Right_Foot': [math.nan, math.nan, math.nan],
         'Total': [math.nan, math.nan]}
        whole_video_COM.append(nan_dict)
    
print(whole_video_COM[46])

{'Head': [0.603276789188385, 0.4252503365278244, 0.0668], 'Trunk': [0.5505917766451836, 0.5005828175127506, 0.4257], 'Left_Upper_Arm': [0.5649352683305741, 0.4886035503566265, 0.0255], 'Right_Upper_Arm': [0.5873992778897286, 0.4100889108121395, 0.0255], 'Left_Forearm': [0.5927355060696602, 0.5246823102474213, 0.0138], 'Right_Forearm': [0.633192343968153, 0.3651664590835571, 0.0138], 'Left_Hand': [0.6404222670356432, 0.5451857108275096, 0.0056], 'Right_Hand': [0.6679719805796941, 0.32760016909639045, 0.0056], 'Left_Thigh': [0.5642293467521667, 0.6191390529155731, 0.1478], 'Right_Thigh': [0.5628450702905655, 0.6140347683668137, 0.1478], 'Left_Shin': [0.609688469696045, 0.7150676864624024, 0.0481], 'Right_Shin': [0.6145293535232544, 0.7127073949813842, 0.0481], 'Left_Foot': [0.6129888494809469, 0.7698694070180258, 0.0129], 'Right_Foot': [0.6246869564056396, 0.77744060754776, 0.0129], 'Total': [0.5697476459131252, 0.5524883709902628]}


In [108]:
type(whole_video_COM[10]['Total'][0])

float

Now we have a block of code that gets all of our COM data from our stored pose estimation results, and can handle errors when the pose estimation couldn't find anything. We'll have to continue to handle those exceptions as we move on to plotting our data, though

Before plotting, let's add all of our functions to our function list to easily port it into a script. Eventually, this function list will end up as a class.

## Function List

In [2]:
### center of mass functions - eventually we would make these a class
def calculateCOM(proximal, distal, com_proximal_multiplier):
    segment_length = distal-proximal
    segment_COM = proximal + (com_proximal_multiplier*segment_length)
    return segment_COM

def calculate_foot_COM(heel, ankle, foot_index):
    COM_foot = (heel + ankle + foot_index)/3
    return COM_foot

def calculate_hand_COM(wrist, index, pinky, com_proximal_multiplier):
    knuckle_width = pinky - index
    Third_metacarple = index + (knuckle_width/3)
    Palm_segment = Third_metacarple - wrist
    COM_hand = wrist + (com_proximal_multiplier*Palm_segment)
    return COM_hand

def calculate_total_COM(COM_Segments, dimension):
    '''Calculate total body COM based on COM of all segments, separately for x and y. 
    Dimension should be 0 for x and 1 for y.'''

    return sum([COM_Segments[segment][2]*COM_Segments[segment][dimension] for segment in COM_Segments])

def handCOM(landmarks):
    # COM of Hands, creating third metacarple point by taking 1/3 of the distance of the segment between index and pinky landmarks
    # then adding that distance to the index. Palm segment will be from wrist joint to 3rd metacarple joint.
    # dictionary values arranged in [wrist, index, pinky, com_proximal_multiplier, com_x, com_y] in that order. 
    Hands = {
        'L_hand' : [landmarks[15], landmarks[19], landmarks[17], 0.7474, 0, 0],
        'R_hand' : [landmarks[16],landmarks[20], landmarks[18], 0.7474, 0, 0],
    }

    for key in Hands:
        #Knuckle_width is defined as the segment from the distal ends of the second to fifth metacarples (index to pinky landmarks)
        wrist_x = Hands[key][0].x
        wrist_y = Hands[key][0].y
        index_x = Hands[key][1].x
        index_y = Hands[key][1].y
        pinky_x = Hands[key][2].x
        pinky_y = Hands[key][2].y
        com_proximal_multiplier = Hands[key][3] 

        COM_hand_x = calculate_hand_COM(wrist_x, index_x, pinky_x, com_proximal_multiplier)
        COM_hand_y = calculate_hand_COM(wrist_y, index_y, pinky_y, com_proximal_multiplier)

        #update Hands dictionary with COM positions
        Hands[key][4] = COM_hand_x
        Hands[key][5] = COM_hand_y
    return Hands

def feetCOM(landmarks):
    #COM of FEET, using the centroid of the triangle formed from heel-ankle-"foot-index" landmarks, arranged in that order in the dictionary values.
    Feet = {
        'L_foot' : [landmarks[29], landmarks[27], landmarks[31], 0, 0],
        'R_foot' : [landmarks[30], landmarks[28], landmarks[32], 0, 0]
    }

    for key in Feet:
        #arguments are ordered heel, ankle, foot index
        COM_foot_x = calculate_foot_COM(Feet[key][0].x, Feet[key][1].x, Feet[key][2].x)
        COM_foot_y = calculate_foot_COM(Feet[key][0].y, Feet[key][1].y, Feet[key][2].y)

        #plot feet COM
        cv2.circle(image, center=tuple(np.multiply((COM_foot_x, COM_foot_y), [width, height]).astype(int)), radius=1, color=(255,0,0), thickness=3)

        #update Feet dictionary with COM positions
        Feet[key][3] = COM_foot_x
        Feet[key][4] = COM_foot_y
        
    return Feet

def trunkCOM(landmarks):
    Trunk = []
    
    #Trunk segment calculations from MidShoulder to Mid Hip. 
    MidShoulder_x = (landmarks[12].x + landmarks[11].x)/2
    MidShoulder_y = (landmarks[12].y + landmarks[11].y)/2
    MidHip_x = (landmarks[24].x + landmarks[23].x)/2
    MidHip_y = (landmarks[24].y + landmarks[23].y)/2
    TrunkCOM_x = calculateCOM(MidShoulder_x, MidHip_x, 0.3782)
    TrunkCOM_y = calculateCOM(MidShoulder_y, MidHip_y, 0.3782)
    
    Trunk.append(TrunkCOM_x)
    Trunk.append(TrunkCOM_y)
    
    return Trunk

def body_segmentsCOM(landmarks):
    #Body Segment Dictionary format: key = body segment, % value = [proximal joint landmark values, distal joint landmark values, COM as a % of segment length]
    Body_Segments = {
        'L_UpperArm' : [landmarks[11] , landmarks[13] , 0.5754, 0, 0],
        'R_UpperArm' : [landmarks[12] , landmarks[14] , 0.5754, 0, 0], 
        'L_Forearm' : [landmarks[13] , landmarks[15] , 0.4559, 0, 0],
        'R_Forearm' : [landmarks[14] , landmarks[16] , 0.4559, 0, 0], 
        'L_Thigh' : [landmarks[23], landmarks[25], 0.3612, 0, 0], 
        'R_Thigh' : [landmarks[24], landmarks[26], 0.3612, 0, 0],
        'L_Shin' : [landmarks[25], landmarks[27], 0.4416, 0, 0],
        'R_Shin' : [landmarks[26], landmarks[28], 0.4416, 0, 0],
        'Head' : [landmarks[7], landmarks[8], 0.5, 0, 0]
    }


    for key in Body_Segments:
        #print(key, 'proximal x ->', Body_Segments[key][0].x) 
        x1 = Body_Segments[key][0].x #proximal joint x value
        y1 = Body_Segments[key][0].y
        x2 = Body_Segments[key][1].x
        y2 = Body_Segments[key][1].y
        com_proximal_multiplier = Body_Segments[key][2]

        COM_x = calculateCOM(x1, x2, com_proximal_multiplier)
        COM_y = calculateCOM(y1, y2, com_proximal_multiplier)
        
        Body_Segments[key][3] = COM_x
        Body_Segments[key][4] = COM_y
        
    return Body_Segments

def define_COM_segments(Body_Segments, Trunk, Feet, Hands):
    COM_Segments = {
                'Head' : [ Body_Segments['Head'][3], Body_Segments['Head'][4], 0.0668],
                'Trunk' : [ Trunk[0], Trunk[1], 0.4257 ],
                'Left_Upper_Arm' : [ Body_Segments['L_UpperArm'][3], Body_Segments['L_UpperArm'][4], 0.0255],
                'Right_Upper_Arm' : [ Body_Segments['R_UpperArm'][3], Body_Segments['R_UpperArm'][4], 0.0255],
                'Left_Forearm' : [ Body_Segments['L_Forearm'][3], Body_Segments['L_Forearm'][4], 0.0138],
                'Right_Forearm' : [ Body_Segments['R_Forearm'][3], Body_Segments['R_Forearm'][4], 0.0138],
                'Left_Hand' : [Hands['L_hand'][4], Hands['L_hand'][5], 0.0056],
                'Right_Hand' : [Hands['R_hand'][4], Hands['R_hand'][5], 0.0056],
                'Left_Thigh' : [Body_Segments['L_Thigh'][3], Body_Segments['L_Thigh'][4], 0.1478],
                'Right_Thigh' : [Body_Segments['R_Thigh'][3], Body_Segments['R_Thigh'][4], 0.1478],
                'Left_Shin' : [Body_Segments['L_Shin'][3], Body_Segments['L_Shin'][4], 0.0481],
                'Right_Shin' : [Body_Segments['R_Shin'][3], Body_Segments['R_Shin'][4], 0.0481],
                'Left_Foot' : [Feet['L_foot'][3], Feet['L_foot'][4], 0.0129], 
                'Right_Foot' : [Feet['R_foot'][3], Feet['R_foot'][4], 0.0129], 
            }
    return COM_Segments

def get_total_COM(COM_Segments):
    COM_total_x = calculate_total_COM(COM_Segments, 0) #0 points to x dimension
    COM_total_y = calculate_total_COM(COM_Segments, 1) #1 points to y dimension
    
    return [COM_total_x, COM_total_y]

def update_COM_Segments(COM_Segments, total_COM):
    COM_Segments['Total'] = total_COM
    
def get_COM_dict(landmarks):
    Hands = handCOM(landmarks)
    Feet = feetCOM(landmarks)
    Trunk = trunkCOM(landmarks)
    Body_Segments = body_segmentsCOM(landmarks)
    COM_Segments = define_COM_segments(Body_Segments, Trunk, Feet, Hands)
    total_COM = get_total_COM(COM_Segments)
    update_COM_Segments(COM_Segments, total_COM)
    return COM_Segments


Now that we have all of our COM data, let's try plotting it!

# Plotting COM Data

We'll need to iterate through our video frames again, while iterating through our COM data, in order to plot everything in order.


In [96]:
#reset capture the video using opencv
cap = cv2.VideoCapture(path)

In [91]:
#setup properties for video writer
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
vid_length = cap.get(cv2.CAP_PROP_FRAME_COUNT)

In [97]:
for frame_idx in range(int(vid_length)): #loop through feed
        ret, frame = cap.read() #getting an image from feed, 'frame' is our video feed variable
        
        #display image - comment out to just save video
        cv2.imshow('COM Display (q to quit)', image)

        if cv2.waitKey(10) & 0xFF == ord('q'): #break out of feed by typing 'q' key
            break

        

#release camera and close all windows
cap.release()